In [2]:
from math import sin, cos, tan
from cmath import exp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
from IPython.display import clear_output
%matplotlib inline

In [3]:
N_layers=10
d_max=350
n1=1.45
n2=2.08
training_structure=np.zeros((N_layers+2,2),dtype=float)
d=np.random.randint(0,high=d_max,size=N_layers+1)

for i in range(1,N_layers+1,2):
    training_structure[i,0]=n1
    training_structure[i,1]=d[i-1]
    training_structure[i+1,0]=n2
    training_structure[i+1,1]=d[i]
training_structure[0,0]=1
training_structure[N_layers+1,0]=1.52
training_structure[0,1]=training_structure[N_layers+1,1]=0
#training_structure=training_structure[::-1]
print(training_structure)

[[  1.     0.  ]
 [  1.45 132.  ]
 [  2.08 214.  ]
 [  1.45  53.  ]
 [  2.08 119.  ]
 [  1.45 142.  ]
 [  2.08 206.  ]
 [  1.45  63.  ]
 [  2.08 273.  ]
 [  1.45 177.  ]
 [  2.08  87.  ]
 [  1.52   0.  ]]


In [6]:
def numpy_N_for_all_angles(n_structure,wl=780, angle_start=20, angle_stop=30, points=20):
    angles=np.linspace(angle_start,angle_stop,points)
    N=np.zeros((len(angles),len(n_structure)),dtype=complex)
    for ind,angle in enumerate(angles):
        N[ind]=(n_structure ** 2 - n_structure[0] ** 2 * (sin(angle/180*np.pi)) ** 2 +0.j) ** 0.5
    return N

def numpy_N_fixed_angle(n_structure,wl_start=400, wl_stop=800, angle=0, points=200):
    return (n_structure ** 2 - n_structure[0] ** 2 * (sin(angle/180*np.pi)) ** 2 +0.j) ** 0.5
    #wavelengths=np.linspace(wl_start,wl_stop,points)
    #N=np.zeros((len(wavelengths),len(n_structure)),dtype=np.complex)
    #for ind,wl in enumerate(wavelengths):

def numpy_create_tm_all_angles(N):
    a=(N[:,1:]+N[:,:-1])/(2.*N[:,1:])
    b=(N[:,1:]-N[:,:-1])/(2.*N[:,1:])
    t=np.zeros((a.shape[0],a.shape[1],2,2),dtype=complex)
    t[:,:,0,0]=t[:,:,1,1]=a
    t[:,:,0,1]=t[:,:,1,0]=b
    return t
 
def numpy_create_tm_fixed_angle(N):
    a=(N[1:]+N[:-1])/(2.*N[1:])
    b=(N[1:]-N[:-1])/(2.*N[1:])
    t=np.zeros((a.shape[0],2,2),dtype=complex)
    t[:,0,0]=t[:,1,1]=a
    t[:,0,1]=t[:,1,0]=b
    return t

def numpy_phase_all_angles(d,N,k0):
    d=d[:,None,:]
    phase=np.zeros((d.shape[0],N.shape[0],d.shape[2],2,2),dtype=complex)
    phase[:,:,:,0,0]=np.exp(1.j * N[:,1:-1]*d * k0)
    phase[:,:,:,1,1]=np.exp(-1.j * N[:,1:-1]*d * k0)  
    return phase

def numpy_phase_all_wavelengths(d,N,k0):
    d=d[:,None,:]
    phase=np.zeros((d.shape[0],k0.shape[0],d.shape[2],2,2),dtype=complex)
    phase[:,:,:,0,0]=np.exp(1.j * N[None, None, 1:-1]*d * k0[None,:,None])
    phase[:,:,:,1,1]=np.exp(-1.j * N[None, None, 1:-1]*d * k0[None,:,None])  
    return phase

def numpy_R_for_all_angles(phase,transfer): 
    
    Total_transfer_matrix = np.zeros((phase.shape[0],phase.shape[1],2,2),dtype=complex)
    Total_transfer_matrix[:,:,0,0]=Total_transfer_matrix[:,:,1,1]=1    
    for i in range(phase.shape[2]):        
        Total_transfer_matrix = np.matmul(transfer[None,:,i], Total_transfer_matrix)
        Total_transfer_matrix = np.matmul(phase[:,:,i], Total_transfer_matrix)        
    Total_transfer_matrix = np.matmul(transfer[None,:,-1], Total_transfer_matrix)
    return -(Total_transfer_matrix[:,:,1, 0] / Total_transfer_matrix[:,:,1, 1])

def numpy_R_for_all_wavelengths(phase,transfer): 
    
    Total_transfer_matrix = np.zeros((phase.shape[0],phase.shape[1],2,2),dtype=complex)
    Total_transfer_matrix[:,:,0,0]=Total_transfer_matrix[:,:,1,1]=1    
    for i in range(phase.shape[2]):        
        Total_transfer_matrix = np.matmul(transfer[None,None,i], Total_transfer_matrix)
        Total_transfer_matrix = np.matmul(phase[:,:,i], Total_transfer_matrix)        
    Total_transfer_matrix = np.matmul(transfer[None,None,-1], Total_transfer_matrix)
    return -(Total_transfer_matrix[:,:,1, 0] / Total_transfer_matrix[:,:,1, 1])

# Var wavelength

In [7]:
n_samples=100000
wavelength_min=600
wavelength_max=800
points=200
thicknes=np.random.randint(0,high=d_max,size=(n_samples,N_layers))

Sample_n_per_cycle=20000
sample_numbers=np.arange(0,stop=n_samples, step=Sample_n_per_cycle)

N_all_wl=numpy_N_fixed_angle(training_structure[:,0])
t_wl=numpy_create_tm_fixed_angle(N_all_wl)

wavelengths=np.linspace(wavelength_min,wavelength_max,points)
wavevector=2*np.pi/wavelengths
training_type="real+imag"
#training_type="abs"
if training_type=="real+imag":
    refl=np.zeros((n_samples,2*points),dtype=float)
    for number in sample_numbers:
        print(number)
        phase=numpy_phase_all_wavelengths(thicknes[number:number+Sample_n_per_cycle],N_all_wl,wavevector)
        total_matrix=numpy_R_for_all_wavelengths(phase,t_wl)
        refl[number:number+Sample_n_per_cycle,:points]=np.real(total_matrix)
        refl[number:number+Sample_n_per_cycle,points:]=np.imag(total_matrix)
else:
    refl=np.zeros((n_samples,points),dtype=float)
    for number in sample_numbers:
        print(number)
        phase=numpy_phase_all_wavelengths(thicknes[number:number+Sample_n_per_cycle],N_all_wl,wavevector)
        total_matrix=numpy_R_for_all_wavelengths(phase,t_wl)
        refl[number:number+Sample_n_per_cycle]=np.abs(total_matrix)

0
20000
40000
60000
80000


In [8]:
header=("structure:\n {} \n wavelength range: {} - {} \n number of points: {}\n type of training:"+training_type).format(
    training_structure[:,0], wavelength_min, wavelength_max,points)
file_for_R='train/train_R_4layers_large.csv'
np.savetxt(file_for_R,refl,
           header=header,delimiter=',')
header2=("structure:\n {} \n maximal layer thickness: {} \n corresponding R file: "+file_for_R
        ).format(training_structure[:,0], d_max)
file_for_thicknes='train/train_thicknes_4layers_large.csv'
np.savetxt(file_for_thicknes,thicknes,
           header=header2,delimiter=',', fmt='%d')

In [9]:
n_samples=40000

thicknes_test=np.random.randint(0,high=d_max,size=(n_samples,N_layers))

Sample_n_per_cycle=20000
sample_numbers=np.arange(0,stop=n_samples, step=Sample_n_per_cycle)
if training_type=="real+imag":
    refl_test=np.zeros((n_samples,2*points),dtype=float)
    for number in sample_numbers:
        print(number)
        phase=numpy_phase_all_wavelengths(thicknes_test[number:number+Sample_n_per_cycle],N_all_wl,wavevector)
        total_matrix=numpy_R_for_all_wavelengths(phase,t_wl)
        refl_test[number:number+Sample_n_per_cycle,:points]=np.real(total_matrix)
        refl_test[number:number+Sample_n_per_cycle,points:]=np.imag(total_matrix)
else:
    refl_test=np.zeros((n_samples,points),dtype=float)
    for number in sample_numbers:
        print(number)
        phase=numpy_phase_all_wavelengths(thicknes_test[number:number+Sample_n_per_cycle],N_all_wl,wavevector)
        total_matrix=numpy_R_for_all_wavelengths(phase,t_wl)
        refl_test[number:number+Sample_n_per_cycle]=np.abs(total_matrix)

0
20000


In [10]:
header=("structure:\n {} \n wavelength range: {} - {} \n number of points: {}\n type of training:"+training_type).format(
    training_structure[:,0], wavelength_min, wavelength_max,points)
file_for_R='test/test_R_4layers_large.csv'
np.savetxt(file_for_R,refl_test,
           header=header,delimiter=',')
header2=("structure:\n {} \n maximal layer thickness: {} \n corresponding R file: "+file_for_R
        ).format(training_structure[:,0], d_max)
file_for_thicknes='test/test_thicknes_4layers_large.csv'
np.savetxt(file_for_thicknes,thicknes_test,
           header=header2,delimiter=',', fmt='%d')

# Var angle

In [7]:
n_samples=200000
angle_min=0
angle_max=40
wl=800
points=100
thicknes=np.random.randint(0,high=d_max,size=(n_samples,N_layers))

Sample_n_per_cycle=20000
sample_numbers=np.arange(0,stop=n_samples, step=Sample_n_per_cycle)

N_all_angles=numpy_N_for_all_angles(training_structure[:,0],wl=wl, angle_start=angle_min, 
                                    angle_stop=angle_max, points=points)
t_angles=numpy_create_tm_all_angles(N_all_angles)

angles=np.linspace(angle_min,angle_max,points)
wavevector=2*np.pi/wl
training_type="real+imag"
#training_type="abs"
if training_type=="real+imag":
    refl=np.zeros((n_samples,2*points),dtype=float)
    for number in sample_numbers:
        print(number)
        phase=numpy_phase_all_angles(thicknes[number:number+Sample_n_per_cycle],N_all_angles,wavevector)
        total_matrix=numpy_R_for_all_angles(phase,t_angles)
        refl[number:number+Sample_n_per_cycle,:points]=np.real(total_matrix)
        refl[number:number+Sample_n_per_cycle,points:]=np.imag(total_matrix)
else:
    refl=np.zeros((n_samples,points),dtype=float)
    for number in sample_numbers:
        print(number)
        phase=numpy_phase_all_angles(thicknes[number:number+Sample_n_per_cycle],N_all_angles,wavevector)
        total_matrix=numpy_R_for_all_angles(phase,t_angles)
        refl[number:number+Sample_n_per_cycle]=np.abs(total_matrix)

0
20000
40000
60000
80000
100000
120000
140000
160000
180000


In [8]:
header=(f"structure:\n {training_structure[:,0]} \n angles range: {angle_min} - {angle_max} \n \
    wavelength:{wl} nm\n number of points: {points}\n type of training:"+training_type)
file_for_R='train/train_R_10layers_angle.csv'
np.savetxt(file_for_R,refl,
           header=header,delimiter=',')
header2=("structure:\n {} \n maximal layer thickness: {} \n corresponding R file: "+file_for_R
        ).format(training_structure[:,0], d_max)
file_for_thicknes='train/train_thicknes_10layers_angle.csv'
np.savetxt(file_for_thicknes,thicknes,
           header=header2,delimiter=',', fmt='%d')

In [9]:
n_samples=40000

thicknes_test=np.random.randint(0,high=d_max,size=(n_samples,N_layers))

Sample_n_per_cycle=20000
sample_numbers=np.arange(0,stop=n_samples, step=Sample_n_per_cycle)
if training_type=="real+imag":
    refl=np.zeros((n_samples,2*points),dtype=float)
    for number in sample_numbers:
        print(number)
        phase=numpy_phase_all_angles(thicknes[number:number+Sample_n_per_cycle],N_all_angles,wavevector)
        total_matrix=numpy_R_for_all_angles(phase,t_angles)
        refl[number:number+Sample_n_per_cycle,:points]=np.real(total_matrix)
        refl[number:number+Sample_n_per_cycle,points:]=np.imag(total_matrix)
else:
    refl=np.zeros((n_samples,points),dtype=float)
    for number in sample_numbers:
        print(number)
        phase=numpy_phase_all_angles(thicknes[number:number+Sample_n_per_cycle],N_all_angles,wavevector)
        total_matrix=numpy_R_for_all_angles(phase,t_angles)
        refl[number:number+Sample_n_per_cycle]=np.abs(total_matrix)

0
20000


In [10]:
header=(f"structure:\n {training_structure[:,0]} \n angles range: {angle_min} - {angle_max} \n \
    wavelength:{wl} nm\n number of points: {points}\n type of training:"+training_type)
file_for_R='test/train_R_10layers_angle.csv'
np.savetxt(file_for_R,refl,
           header=header,delimiter=',')
header2=("structure:\n {} \n maximal layer thickness: {} \n corresponding R file: "+file_for_R
        ).format(training_structure[:,0], d_max)
file_for_thicknes='test/train_thicknes_10layers_angle.csv'
np.savetxt(file_for_thicknes,thicknes,
           header=header2,delimiter=',', fmt='%d')